In [ ]:
import numpy as np
from numpy import shape

from datetime import date

import matplotlib.pyplot as plt

from datetime import datetime
from pytz import timezone

In [ ]:
%run '/home/jovyan/rna/Python_Code/functions.py'

##### Import MNIST data

In [ ]:
# Import data from sklearn

# # MNIST von sklearn mit 28x28 Pixel
# from sklearn.datasets import fetch_openml

# # Load data from https://www.openml.org/d/554
# X_raw, y_raw = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [ ]:
# Import data that was saved on this system

raw_data = load_file(file = "/home/jovyan/rna/MNIST/Raw_Data/MNIST_images.pkl")
X_raw = raw_data[0]
y_raw = raw_data[1]

In [ ]:
# Filter on indices of '0' and '8'
indices = (y_raw == '0') | (y_raw == '8')

# reshape into squares
X = X_raw[indices, :]
y = y_raw[indices]

### Load correlation matrix $\bar{Z}$

In [ ]:
corr_matrix = np.genfromtxt('/home/jovyan/rna/MNIST/Distance_Correlation_Matrix/corr_matrix_500_2022-05-16.csv',
                         delimiter=',',)[1:785, 1:785] # remove column- and rownames

### Distanzmatrix $M = 1 - \bar{Z}$

In [ ]:
M = 1 - corr_matrix

M_sqr = np.square(M)

### Calculation of all MNIST persistences

In [ ]:
# number of images
n_images = len(X)

# Größe der Subsamples
n_subsample = 150

n_persistences_per_subject = 300

# Folderpath where the persistences will be saved
folderpath = "/home/jovyan/rna/MNIST/Persistences_from_filtration_all_images//"
        
#------------------------------------------------------------
#-----------------Calculation of persistences----------------
#------------------------------------------------------------

for index_image in range(n_images):
    
    msg = ["Berechnung für Bild ", index_image, "| Uhrzeit: ", datetime.now(tz = timezone('Europe/Berlin')).strftime("%Y_%m_%d-%I:%M:%S_%p")]
    %store msg >>/home/jovyan/progress_persistences_mnist_2022_05_25.txt
    
    print("Berechnung t_sigma")
    t_sigma_subject = calc_t_sigma(sample_index = index_image,
                                   M_sqr = M_sqr,
                                   X_sqr = X_sqr,
                                   M = M)
    
    persistence_subject = calc_persistence_oneperson(t_sigma_subject,
                                                     n_subsample,
                                                     n_persistences_per_subject)
    
    filepath = folderpath + "persistences_image_" + str(index_image).zfill(5) + "_300_2022_05_25.pkl"
    save_file(filepath, persistence_subject)

##### Speichern aller Persistenzen

In [ ]:
# Bessere Variante mit ausgesondertem Folderpath
folderpath = "/home/jovyan/rna/MNIST/Persistences_from_filtration/"

for i in range(len(persistence_all_subjects)):

    filepath = folderpath + "persistences_image" + str(i).zfill(5) + "_300_2022_05_18.pkl"
    save_file(filepath, persistence_all_subjects[i])

##### Einlesen der gespeicherten Persistenzen

In [ ]:
# Überarbeitete Variante mit separatem folderpath

folderpath = "/home/jovyan/rna/MNIST/Persistences_from_filtration/"
test = []

for i in range(383):
    filepath = folderpath + "persistences_all_subjects_" + str(i).zfill(5) + "_300_2022_02_23.pkl"
    
    with open(filepath, 'rb') as f:
        tmp = dill.load(f)
    test.insert(i, tmp)